In [ ]:
!pip install tslearn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 17.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from tslearn.preprocessing import TimeSeriesScalerMeanVariance
#from tslearn.clustering import TimeSeriesKMeans
from joblib import Parallel, delayed
#from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
import pickle

In [ ]:
path_products = './datasets/tb_productos.txt'
path_stocks = './datasets/tb_stocks.txt'
path_sells = './datasets/sell-in.txt'
path_products_to_predict = './datasets/product_id_apredecir201912.txt'

# 📁 Crear carpeta datasets (si no existe)
!mkdir -p datasets

# 📥 Descargar archivos desde la URL pública
!wget -q https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_productos.txt -O ./datasets/tb_productos.txt
!wget -q https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_stocks.txt -O ./datasets/tb_stocks.txt
!wget -q https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/sell-in.txt.gz -O ./datasets/sell-in.txt.gz
!wget -q https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt -O ./datasets/product_id_apredecir201912.txt

# 🗜️ Descomprimir el archivo sell-in.txt.gz
import gzip
import shutil

with gzip.open('./datasets/sell-in.txt.gz', 'rb') as f_in:
    with open('./datasets/sell-in.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("✅ Archivos descargados y preparados correctamente.")

✅ Archivos descargados y preparados correctamente.


In [ ]:
tn_scaler_path='./scalers/scalers.pkl'
encoders_path='./encoders/encoders.pkl'
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size']

df_products = pd.read_csv(path_products, sep='\t', dtype={ 'product_id': str, 'sku_size': str })
df_stocks = pd.read_csv(path_stocks, sep='\t', dtype={ 'product_id': str, 'periodo': str })
df_sellin = pd.read_csv(path_sells, sep='\t', dtype={ 'product_id': str, 'customer_id': str, 'periodo': str })
df_products_to_predict = pd.read_csv(path_products_to_predict, dtype={ 'product_id': str })



In [ ]:
# 🧹 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [ ]:
# 📄 Leer lista de productos a predecir
with open("./datasets/product_id_apredecir201912.txt", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

# prompt: como aplico esto, a mi script
# # 📄 Leer lista de productos a predecir
# with open("product_id_apredecir201912.TXT", "r") as f:
#     product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

# 📄 Leer lista de productos a predecir
with open(path_products_to_predict, "r") as f:
    product_ids_to_predict = [int(line.strip()) for line in f if line.strip().isdigit()]

# Filtrar df_sellin para incluir solo los product_ids_to_predict
df_sellin_filtered = df_sellin[df_sellin['product_id'].astype(int).isin(product_ids_to_predict)].copy()


In [ ]:
df_products

,cat1,cat2,cat3,brand,sku_size,product_id,descripcion
0,FOODS,ADEREZOS,Aji Picante,NATURA,240,20609,Salsa Aji Picante
1,FOODS,ADEREZOS,Barbacoa,NATURA,250,20266,Salsa Barbacoa
2,FOODS,ADEREZOS,Barbacoa,NATURA,400,20325,Salsa Barbacoa
3,FOODS,ADEREZOS,Barbacoa,NATURA,500,20503,Salsa Barbacoa
4,FOODS,ADEREZOS,Chimichurri,NATURA,350,20797,Chimichurri
...,...,...,...,...,...,...,...
1246,REF,TE,Frutas,TWININGS,20,21271,Frutas
1247,REF,TE,Hierbas,TWININGS,20,21202,Manzanilla
1248,REF,TE,Hierbas,TWININGS,20,21218,Menta
1249,REF,TE,Verde,TWININGS,20,21192,Verde


In [ ]:
df_products_to_predict

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005
...,...
775,21263
776,21265
777,21266
778,21267


In [ ]:
# Filtrar por período 201912
df_201912 = df_sellin[df_sellin['periodo'] == "201912"]

In [ ]:
df_agrupado = df_201912.groupby('product_id')['tn'].sum().reset_index()

In [ ]:
df_201912['product_id'] = df_201912['product_id'].astype(str)
df_products_to_predict['product_id'] = df_products_to_predict['product_id'].astype(str)

/tmp/ipython-input-11-2934532088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_201912['product_id'] = df_201912['product_id'].astype(str)


In [ ]:
productos_filtrados = df_201912[df_201912['product_id'].isin(df_products_to_predict['product_id'])]

In [ ]:
df_sellin.columns

Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'timestamp'],
      dtype='object')

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Asegurar que product_id es str en todos
df_sellin['product_id'] = df_sellin['product_id'].astype(str)
df_products['product_id'] = df_products['product_id'].astype(str)
df_products_to_predict['product_id'] = df_products_to_predict['product_id'].astype(str)

# Paso 1: Agregamos tn por product_id y periodo
df_agg = df_sellin.groupby(['product_id', 'periodo'], as_index=False)['tn'].sum()
df_agg = df_agg.sort_values(['product_id', 'periodo'])
df_agg['periodo'] = df_agg['periodo'].astype(int)


In [ ]:

# Paso 2: Calcular lags y clase
for i in range(1, 12):
    df_agg[f'tn_{i}'] = df_agg.groupby('product_id')['tn'].shift(i)

df_agg['clase'] = df_agg.groupby('product_id')['tn'].shift(-2)

# Paso 3: Merge con catálogo
df_agg = df_agg.merge(df_products, on='product_id', how='left')

In [ ]:
df_agg

,product_id,periodo,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,...,tn_9,tn_10,tn_11,clase,cat1,cat2,cat3,brand,sku_size,descripcion
0,20001,201701,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
1,20001,201702,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
2,20001,201703,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
3,20001,201704,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1520.06539,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
4,20001,201705,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,...,NaN,NaN,NaN,1030.67391,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31238,21295,201701,0.00699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31239,21296,201708,0.00651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31240,21297,201701,0.00579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31241,21298,201708,0.00573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
num_distinct_products = df_agg['product_id'].nunique()
print(f"Number of distinct product_ids in df_agg: {num_distinct_products}")

Number of distinct product_ids in df_agg: 1233


In [ ]:
# prompt: como hago para filtrar en df_agg para los product_id que estan en df_products_to_predict

df_agg_filtered = df_agg[df_agg['product_id'].isin(df_products_to_predict['product_id'])]
df_agg_filtered

,product_id,periodo,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,...,tn_9,tn_10,tn_11,clase,cat1,cat2,cat3,brand,sku_size,descripcion
0,20001,201701,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
1,20001,201702,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
2,20001,201703,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
3,20001,201704,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1520.06539,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
4,20001,201705,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,...,NaN,NaN,NaN,1030.67391,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31206,21276,201908,0.01265,0.00223,0.04086,0.09283,0.10173,0.12249,NaN,NaN,...,NaN,NaN,NaN,0.02079,PC,PIEL1,Cara,NIVEA,140,reconstruccion
31207,21276,201909,0.01856,0.01265,0.00223,0.04086,0.09283,0.10173,0.12249,NaN,...,NaN,NaN,NaN,0.03341,PC,PIEL1,Cara,NIVEA,140,reconstruccion
31208,21276,201910,0.02079,0.01856,0.01265,0.00223,0.04086,0.09283,0.10173,0.12249,...,NaN,NaN,NaN,0.00892,PC,PIEL1,Cara,NIVEA,140,reconstruccion
31209,21276,201911,0.03341,0.02079,0.01856,0.01265,0.00223,0.04086,0.09283,0.10173,...,NaN,NaN,NaN,NaN,PC,PIEL1,Cara,NIVEA,140,reconstruccion


In [ ]:
num_distinct_products = df_agg_filtered['product_id'].nunique()
print(f"Number of distinct product_ids in df_agg: {num_distinct_products}")

Number of distinct product_ids in df_agg: 780


In [ ]:

# Paso 4: Definir mágicos y armar dataset de entrenamiento
magicos = ["20002", "20003", "20006", "20010", "20011", "20018", "20019", "20021",
           "20026", "20028", "20035", "20039", "20042", "20044", "20045", "20046", "20049",
           "20051", "20052", "20053", "20055", "20008", "20001", "20017", "20086", "20180",
           "20193", "20320", "20532", "20612", "20637", "20807", "20838"]


In [ ]:

campos = ['tn'] + [f'tn_{i}' for i in range(1, 12)] + ['clase']
train_df = df_agg_filtered[(df_agg_filtered['periodo'] == 201812) & (df_agg_filtered['product_id'].isin(magicos))]
train_df = train_df.dropna(subset=campos)


In [ ]:
train_df

,product_id,periodo,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,...,tn_9,tn_10,tn_11,clase,cat1,cat2,cat3,brand,sku_size,descripcion
23,20001,201812,1486.68669,1813.01511,2295.19832,1438.67455,1800.96168,1470.41009,1150.79169,1293.89788,...,1856.83534,1043.76470,1169.07532,1259.09363,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
59,20002,201812,1009.45458,1766.81068,1378.49032,954.23575,1161.88430,977.40239,1033.82845,1103.39191,...,966.86044,712.00087,984.80167,1043.01349,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,Maquina 1er lavado
95,20003,201812,769.82869,1206.91773,1313.34211,912.34156,955.97079,656.22700,660.73323,784.35885,...,778.55594,788.30749,907.56304,758.32657,FOODS,ADEREZOS,Mayonesa,NATURA,475,Regular sin TACC
203,20006,201812,407.75925,566.66809,513.15472,478.04388,615.70617,515.20419,468.15260,865.28861,...,862.19361,588.56272,470.33785,479.99914,HC,VAJILLA,Cristalino,LIMPIEX,750,Desengrasante cristal Limon
275,20008,201812,426.32899,433.50170,532.45644,436.96269,554.82147,526.38149,554.57063,707.59267,...,765.98901,506.25385,469.29224,476.98787,HC,VAJILLA,Opaco,LIMPIEX,750,Desengrasante crema glicerina
347,20010,201812,285.02947,414.97753,612.50721,480.60235,582.83104,331.96807,223.87746,227.24082,...,653.77607,477.48363,298.25586,337.76009,HC,ROPA LAVADO,Polvo,LIMPIEX,400,Mano con suavizante
383,20011,201812,321.09714,289.13976,177.75576,189.59850,191.07270,300.26178,437.75550,484.04538,...,526.99374,601.26066,340.75314,431.62938,HC,ROPA LAVADO,Polvo,LIMPIEX,3000,Maquina con suavizante
599,20017,201812,259.32724,286.83676,331.23254,288.35292,374.95908,351.60065,316.45841,533.53335,...,488.79258,377.84497,291.70926,308.71060,HC,VAJILLA,Opaco,LIMPIEX,750,Desengrasante crema colageno
635,20018,201812,326.01506,371.52958,161.58557,282.43485,375.61778,325.03223,420.33781,388.43687,...,510.33171,337.54792,342.16945,265.84135,HC,VAJILLA,Cristalino,LIMPIEX,750,Desengrasante cristal Aloe Vera
671,20019,201812,446.69747,532.98143,552.71975,417.95455,387.73155,351.05610,262.33076,356.42982,...,321.26878,629.89543,243.71984,323.66178,FOODS,ADEREZOS,Mayonesa,NATURA,950,Regular sin TACC


In [ ]:

# Paso 5: Entrenar modelo
X_train = train_df[['tn'] + [f'tn_{i}' for i in range(1, 12)]]
y_train = train_df['clase']

In [ ]:
df_check = df_agg_filtered[(df_agg_filtered['periodo'] == "201812") & (df_agg_filtered['product_id'].isin(magicos))]
print("Filas encontradas antes del dropna:", df_check.shape[0])
print(df_check[campos].isna().sum())


Filas encontradas antes del dropna: 0
tn       0
tn_1     0
tn_2     0
tn_3     0
tn_4     0
tn_5     0
tn_6     0
tn_7     0
tn_8     0
tn_9     0
tn_10    0
tn_11    0
clase    0
dtype: int64


In [ ]:

modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Paso 6: Predecir para periodo 201901
productos_780 = df_products_to_predict['product_id'].tolist()
df_pred = df_agg_filtered[(df_agg_filtered['periodo'] == 201901) & (df_agg_filtered['product_id'].isin(productos_780))].copy()

campos_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
completos = df_pred.dropna(subset=campos_tn).copy()
incompletos = df_pred[~df_pred.index.isin(completos.index)].copy()

X_completos = completos[campos_tn]
completos['pred_clase'] = modelo.predict(X_completos)
promedio_pred = completos['pred_clase'].mean()
incompletos['pred_clase'] = promedio_pred

resultado_final = pd.concat([completos, incompletos]).sort_index()

In [ ]:
coef_df = pd.DataFrame({
    'Variable': ['tn'] + [f'tn_{i}' for i in range(1, 12)],
    'Coeficiente': modelo.coef_
})
print("Intercepto:", modelo.intercept_)
print(coef_df)


Intercepto: 0.44146718354036807
   Variable  Coeficiente
0        tn    -0.001339
1      tn_1     0.236558
2      tn_2     0.178208
3      tn_3    -0.060031
4      tn_4    -0.161875
5      tn_5    -0.007775
6      tn_6     0.151936
7      tn_7     0.043933
8      tn_8     0.142839
9      tn_9     0.103804
10    tn_10     0.119211
11    tn_11     0.073671


In [ ]:
# Paso 1: Filtrar productos a predecir en periodo 201912
df_pred_201912 = df_agg_filtered[(df_agg_filtered['periodo'] == 201912) & (df_agg_filtered['product_id'].isin(productos_780))].copy()

# Paso 2: Separar completos e incompletos
campos_tn = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
completos_201912 = df_pred_201912.dropna(subset=campos_tn).copy()
incompletos_201912 = df_pred_201912[~df_pred_201912.index.isin(completos_201912.index)].copy()

# Paso 3: Predecir con modelo ya entrenado (coeficientes ya calculados)
X_completos_201912 = completos_201912[campos_tn]
completos_201912['pred_clase'] = modelo.predict(X_completos_201912)

# Paso 4: Calcular promedio de las predicciones
promedio_pred_201912 = completos_201912['pred_clase'].mean()

# Paso 5: Aplicar promedio a incompletos
# Para cada SKU incompleto, buscamos su promedio histórico
def promedio_historico(sku):
    historico = df_agg_filtered[(df_agg_filtered['product_id'] == sku) & (df_agg_filtered['periodo'] < 201912)]
    return historico['tn'].mean()

# Aplicamos la función fila por fila
incompletos_201912['pred_clase'] = incompletos_201912['product_id'].apply(promedio_historico)


# Paso 6: Unir y agregar período destino
resultado_201912 = pd.concat([completos_201912, incompletos_201912]).sort_index()
resultado_201912['periodo_predicho'] = 202002

In [ ]:
# Toneladas reales (tn) en 201912 para los 780 productos
real_201912 = df_agg_filtered[(df_agg_filtered['periodo'] == 201912) & (df_agg_filtered['product_id'].isin(productos_780))].copy()

# Separar completos e incompletos
completos_real = real_201912.dropna(subset=campos_tn).copy()
incompletos_real = real_201912[~real_201912.index.isin(completos_real.index)].copy()

# Mostrar resultados
print("Toneladas REALES en 201912 (como en la tabla del profe):")
print(" - Completos:", round(completos_real['tn'].sum(), 2))
print(" - Incompletos:", round(incompletos_real['tn'].sum(), 2))
print(" - Total:", round(real_201912['tn'].sum(), 2))


Toneladas REALES en 201912 (como en la tabla del profe):
 - Completos: 23447.86
 - Incompletos: 1697.39
 - Total: 25145.25


In [ ]:
# Mostrar verificación al estilo del profesor
print("Resumen predicción para periodo 202002 (usando datos de 201912):")
print("---------------------------------------------------------------")
print("Registros totales:", resultado_201912.shape[0])
print("Completos:", completos_201912.shape[0])
print("Incompletos:", incompletos_201912.shape[0])
print()
print("Toneladas predichas:")
print(" - Completos:", round(completos_201912['pred_clase'].sum(), 2))
print(" - Incompletos:", round(incompletos_201912['pred_clase'].sum(), 2))
print(" - Total:", round(resultado_201912['pred_clase'].sum(), 2))

Resumen predicción para periodo 202002 (usando datos de 201912):
---------------------------------------------------------------
Registros totales: 780
Completos: 656
Incompletos: 124

Toneladas predichas:
 - Completos: 25785.02
 - Incompletos: 2304.94
 - Total: 28089.96


In [ ]:
# Formato final: solo product_id y tn
export_kaggle = resultado_201912[['product_id', 'pred_clase']].copy()
export_kaggle = export_kaggle.rename(columns={'pred_clase': 'tn'})

# Exportar a CSV sin índice
export_kaggle.to_csv('prediccion_regresion2años.csv', index=False)

print("Archivo 'prediccion_regresion1años.csv' generado en formato correcto para Kaggle.")

Archivo 'prediccion_regresion1años.csv' generado en formato correcto para Kaggle.


HASTA ACA LA REGRESION, PROBAMOS AUTOGULON

In [ ]:
# 💬 Instalar AutoGluon si es necesario
%pip install autogluon.timeseries

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.4/382.4 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
df_sellin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 8 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   periodo                object        
 1   customer_id            object        
 2   product_id             object        
 3   plan_precios_cuidados  int64         
 4   cust_request_qty       int64         
 5   cust_request_tn        float64       
 6   tn                     float64       
 7   timestamp              datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 179.8+ MB


In [ ]:
# Asegurar que los IDs sean strings
product_ids = [str(p) for p in product_ids]

# Filtro hasta fin de diciembre 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-31') &
    (df_sellin['product_id'].isin(product_ids))
]


In [ ]:
df_filtered

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,timestamp
0,201701,10234,20524,0,2,0.05300,0.05300,2017-01-01
1,201701,10032,20524,0,1,0.13628,0.13628,2017-01-01
2,201701,10217,20524,0,1,0.03028,0.03028,2017-01-01
3,201701,10125,20524,0,1,0.02271,0.02271,2017-01-01
4,201701,10012,20524,0,11,1.54452,1.54452,2017-01-01
...,...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230,2019-12-01
2945814,201912,10092,20853,0,1,0.00669,0.00669,2019-12-01
2945815,201912,10006,20853,0,7,0.02898,0.02898,2019-12-01
2945816,201912,10018,20853,0,4,0.01561,0.01561,2019-12-01


In [ ]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [ ]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [ ]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [ ]:
# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [ ]:
# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

In [ ]:
ts_data

,,product_id,tn
item_id,timestamp,,
20001,2017-01-01,20001,934.77222
20002,2017-01-01,20002,550.15707
20003,2017-01-01,20003,1063.45835
20004,2017-01-01,20004,555.91614
20005,2017-01-01,20005,494.27011
...,...,...,...
21263,2019-12-01,21263,0.01270
21265,2019-12-01,21265,0.05007
21266,2019-12-01,21266,0.05121


In [ ]:
# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start),
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to '/content/AutogluonModels/ag-20250720_181128'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       10.73 GB / 12.67 GB (84.6%)
Disk Space Avail:   64.83 GB / 107.72 GB (60.2%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 3600,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to frequency 'MS'.
Provi

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/821M [00:00<?, ?B/s]

	-0.1905       = Validation score (-WQL)
	34.60   s     = Training runtime
	14.30   s     = Validation (prediction) runtime
Training timeseries model ChronosFineTuned[bolt_small]. Training for up to 570.6s of the 3423.5s of remaining time.
	Skipping covariate_regressor since the dataset contains no covariates or static features.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/191M [00:00<?, ?B/s]

	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	Saving fine-tuned model to /content/AutogluonModels/ag-20250720_181128/models/ChronosFineTuned[bolt_small]/W0/fine-tuned-ckpt
	Skipping covariate_regressor since the dataset contains no covariates or static features.
	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	Saving fine-tuned model to /content/AutogluonModels/ag-20250720_181128/models/ChronosFineTuned[bolt_small]/W1/fine-tuned-ckpt
	-0.1819       = Validation score (-WQL)
	505.81  s     = Training runtime
	3.07    s     = Validation (prediction) runtime
Training timeseries model TemporalFusionTransformer. Training for up to 582.9s of the 2914.6s of remaining time.
	-0.1957       = Validation score (-WQL)
	412.96  s     = Training runtime
	0.80    s     = Validation (prediction) runtime
Training timeseries model DeepAR. Training for up to 633.6s of the 2500.8s of remaining time.
	-0.1860    

In [ ]:
# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [ ]:
# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

Index(['item_id', 'timestamp', 'mean'], dtype='object')


In [ ]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

In [ ]:
# 💾 7. Guardar archivo
resultado.to_csv("predicciones_autogulon basico.csv", index=False)
resultado.head()

,product_id,tn
1,20001,1314.048807
3,20002,1085.482960
5,20003,683.597879
7,20004,519.754406
9,20005,507.141687


In [ ]:
# Asegurarse de que timestamp sea una columna y no índice
ts_data = ts_data.reset_index()  # Si ya no lo está
df_products_renamed = df_products.rename(columns={'product_id': 'item_id'})
ts_data_enriched = ts_data.merge(df_products_renamed, on='item_id', how='left')


In [ ]:
ts_data_enriched

,item_id,timestamp,product_id,tn,cat1,cat2,cat3,brand,sku_size,descripcion
0,20001,2017-01-01,20001,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
1,20002,2017-01-01,20002,550.15707,HC,ROPA LAVADO,Liquido,LIMPIEX,3000,Maquina 1er lavado
2,20003,2017-01-01,20003,1063.45835,FOODS,ADEREZOS,Mayonesa,NATURA,475,Regular sin TACC
3,20004,2017-01-01,20004,555.91614,FOODS,ADEREZOS,Mayonesa,NATURA,240,Regular sin TACC
4,20005,2017-01-01,20005,494.27011,FOODS,ADEREZOS,Mayonesa,NATURA,120,Regular sin TACC
...,...,...,...,...,...,...,...,...,...,...
22344,21263,2019-12-01,21263,0.01270,PC,CABELLO,SHAMPOO,VICHY,250,Sabor 1
22345,21265,2019-12-01,21265,0.05007,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32,mascara facial 2
22346,21266,2019-12-01,21266,0.05121,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32,mascara facial 1
22347,21267,2019-12-01,21267,0.01569,PC,PIEL1,Cara,NIVEA,250,Limpieza Facial


In [ ]:
ts_data_enriched.columns
# ['item_id', 'timestamp', 'tn', 'cat1', 'cat2', ..., 'descripcion']


Index(['item_id', 'timestamp', 'product_id', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion'],
      dtype='object')

In [ ]:
# 1. Filtrar datos hasta dic-2019
ts_final_train = ts_data_enriched[ts_data_enriched['timestamp'] <= '2019-12-01']

# 2. Establecer índice jerárquico
ts_final_train = ts_final_train.set_index(['item_id', 'timestamp']).sort_index()


In [ ]:
ts_final_train

product_id          tn cat1         cat2     cat3  brand  \
item_id timestamp                                                             
20001   2017-01-01      20001   934.77222   HC  ROPA LAVADO  Liquido  ARIEL   
        2017-02-01      20001   798.01620   HC  ROPA LAVADO  Liquido  ARIEL   
        2017-03-01      20001  1303.35771   HC  ROPA LAVADO  Liquido  ARIEL   
        2017-04-01      20001  1069.96130   HC  ROPA LAVADO  Liquido  ARIEL   
        2017-05-01      20001  1502.20132   HC  ROPA LAVADO  Liquido  ARIEL   
...                       ...         ...  ...          ...      ...    ...   
21276   2019-08-01      21276     0.01265   PC        PIEL1     Cara  NIVEA   
        2019-09-01      21276     0.01856   PC        PIEL1     Cara  NIVEA   
        2019-10-01      21276     0.02079   PC        PIEL1     Cara  NIVEA   
        2019-11-01      21276     0.03341   PC        PIEL1     Cara  NIVEA   
        2019-12-01      21276     0.00892   PC        PIEL1     Cara  NIVEA   

                   sku_size     descripcion  
item_id timestamp                            
20001   2017-01-01     3000          genoma  
        2017-02-01     3000          genoma  
        2017-03-01     3000          genoma  
        2017-04-01     3000          genoma  
        2017-05-01     3000          genoma  
...                     ...             ...  
21276   2019-08-01      140  reconstruccion  
        2019-09-01      140  reconstruccion  
        2019-10-01      140  reconstruccion  
        2019-11-01      140  reconstruccion  
        2019-12-01      140  reconstruccion  

[22349 rows x 8 columns]

In [ ]:

# 3. Entrenar predictor
predictor_final = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'
)

predictor_final.fit(
    ts_final_train,
    num_val_windows=4,
    presets='best_quality',
    time_limit=2*60*60
)



Beginning AutoGluon training... Time limit = 7200s
AutoGluon will save models to '/content/AutogluonModels/ag-20250720_184421'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       8.99 GB / 12.67 GB (71.0%)
Disk Space Avail:   63.42 GB / 107.72 GB (58.9%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 4,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 7200,
 'verbosity': 2}

train_data with frequency 'IRREG' has been re

In [ ]:

# 3. Predecir ene-feb 2020
forecast_final = predictor_final.predict(ts_final_train)
forecast_df = forecast_final['mean'].reset_index()

# 4. Filtrar solo febrero 2020
forecast_feb2020 = forecast_df[forecast_df['timestamp'] == '2020-02-01']
forecast_feb2020 = forecast_feb2020.rename(columns={'mean': 'tn', 'item_id': 'product_id'})

# 5. Exportar resultados
forecast_feb2020[['product_id', 'tn']].to_csv('predicciones_febrero2020_autogluon.csv', index=False)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [ ]:
from autogluon.timeseries import TimeSeriesPredictor
import numpy as np

# 1. Preprocesamiento: casteo y transformación log1p
ts_data_transformed = ts_data_enriched.copy()
for col in ['cat1', 'cat2', 'cat3', 'brand']:
    ts_data_transformed[col] = ts_data_transformed[col].astype('category')

ts_data_transformed['tn'] = np.log1p(ts_data_transformed['tn'])

# 2. Entrenar hasta dic-2019
ts_final_train = ts_data_transformed[ts_data_transformed['timestamp'] <= '2019-12-31']
ts_final_train = ts_final_train.set_index(['item_id', 'timestamp']).sort_index()

# 3. Entrenar modelo avanzado
predictor_advanced = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS',
    eval_metric='WAPE'
)


In [ ]:
# 3. Entrenar modelo avanzado
predictor_advanced = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS',
    eval_metric='WAPE'
)

predictor_advanced.fit(
    ts_final_train,
    hyperparameters={
        'DeepAR': {},
        'ETS': {},
        'Naive': {}
    },
    num_val_windows=5,
    time_limit=2*60*60
)

Beginning AutoGluon training... Time limit = 7200s
AutoGluon will save models to '/content/AutogluonModels/ag-20250720_200744-001'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       9.71 GB / 12.67 GB (76.6%)
Disk Space Avail:   62.68 GB / 107.72 GB (58.2%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WAPE,
 'freq': 'MS',
 'hyperparameters': {'DeepAR': {}, 'ETS': {}, 'Naive': {}},
 'known_covariates_names': [],
 'num_val_windows': 5,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 7200,
 'verbosity': 2}

train_data with frequency 'IRREG' has been r

In [ ]:

# 4. Predecir ene-feb 2020 y revertir log1p
forecast = predictor_advanced.predict(ts_final_train)
forecast_df = forecast['mean'].reset_index()
forecast_df['forecast'] = np.expm1(forecast_df['mean'])

# 5. Filtrar febrero 2020
forecast_feb2020 = forecast_df[forecast_df['timestamp'] == '2020-02-01']
forecast_feb2020 = forecast_feb2020.rename(columns={'item_id': 'product_id'})

# 6. Exportar resultados
forecast_feb2020[['product_id', 'forecast']].rename(columns={'forecast': 'tn'}) \
    .to_csv('predicciones_febrero2020_autogluon_avanzado.csv', index=False)


In [ ]:
from google.colab import files
files.download('predicciones_febrero2020_autogluon_avanzado.csv')
